# NO2 Data Aggregation - Plots

Aggregates NO2 values by administrative boundaries and a regular hexagon grid.

To convert from mol/m<sup>2</sup> to molecules/cm<sup>2</sup>, multiply by 6.022140857e+19 (6.022140857 * 10<sup>19</sup>).

# Environment Setup

In [ ]:
# Load Notebook formatter
%load_ext nb_black
# %reload_ext nb_black

In [ ]:
# Import packages
import os
import glob
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import pandas as pd
import geopandas as gpd
import rasterio as rio
import rasterstats as rs
import radiance as rd
import sentinel as stl

In [ ]:
# Set Options
# sns.set(font_scale=1.5, style="whitegrid")
sns.set(font_scale=1.5)
pd.set_option("display.max_columns", None)
pd.set_option("precision", 15)

In [ ]:
# Set working directory
os.chdir("..")
print(f"Working directory: {os.getcwd()}")

# User-Defined Variables

In [ ]:
# Set paths to Singapore shapefiles at levels 0 and 1, and hexagon grid
singapore_level_0_path = os.path.join(
    "02-raw-data", "vector", "singapore", "gadm36_singapore.shp"
)

singapore_level_1_path = os.path.join(
    "02-raw-data", "vector", "singapore", "gadm36_singapore_level_1.shp"
)

singapore_hexagon_grid_path = os.path.join(
    "03-processed-data",
    "vector",
    "singapore",
    "singapore_hexagon_grid_0p01.shp",
)

# Set path to Malaysia levels 0 and 1 shapefile
malaysia_level_0_path = os.path.join(
    "02-raw-data", "vector", "malaysia", "gadm36_malaysia.shp"
)

malaysia_level_1_path = os.path.join(
    "02-raw-data", "vector", "malaysia", "gadm36_malaysia_level_1.shp"
)

In [ ]:
# Set paths to aggregated data shapfiles
aggregate_level_1_path = os.path.join(
    "03-processed-data",
    "vector",
    "singapore",
    "aggregate_level_1_jul_2018_jul_2020.shp",
)

aggregate_hexagon_path = os.path.join(
    "03-processed-data",
    "vector",
    "singapore",
    "aggregate_hexagon_grid_jul_2018_jul_2020.shp",
)

In [ ]:
# Set folder to NO2 rasters
no2_folder = os.path.join(
    "03-processed-data", "raster", "singapore", "statistics", "monthly",
)

# Data Acquisition and Preprocessing

In [ ]:
# Get geodataframes with geometry only (for plotting)
singapore_level_0_geometry = stl.get_geometry(singapore_level_0_path)
singapore_level_1_geometry = stl.get_geometry(singapore_level_1_path)
singapore_hexagon_grid_geometry = stl.get_geometry(singapore_hexagon_grid_path)
malaysia_level_0_geometry = stl.get_geometry(malaysia_level_0_path)
malaysia_level_1_geometry = stl.get_geometry(malaysia_level_1_path)

In [ ]:
# Get all NO2 means into list
no2_means = sorted(glob.glob(os.path.join(no2_folder, "*MEAN*tif")))

# Extract plotting extent
singapore_extent = stl.extract_plotting_extent(no2_means[0])

# Create extent polygon
singapore_polygon = stl.create_polygon_from_extent(singapore_extent)

# Clip Malaysia to Singapore extent polygon
malaysia_level_0_clipped = gpd.clip(
    gpd.GeoDataFrame(
        geometry=[singapore_polygon], crs=malaysia_level_0_geometry.crs
    ),
    malaysia_level_0_geometry,
)

malaysia_level_1_clipped = gpd.clip(
    gpd.GeoDataFrame(
        geometry=[singapore_polygon], crs=malaysia_level_1_geometry.crs
    ),
    malaysia_level_1_geometry,
)

In [ ]:
# Read aggregated shapefiles into geodataframes
singapore_level_1_gdf = gpd.read_file(aggregate_level_1_path)
hexagon_grid_gdf = gpd.read_file(aggregate_hexagon_path)

In [ ]:
# Restore original column names (shorted by export to shapefile)
level_1_gdf = singapore_level_1_gdf.rename(
    columns={
        # Magnitude
        "change_jul": "change_jul_2018_2019_magnitude",
        "change_aug": "change_aug_2018_2019_magnitude",
        "change_sep": "change_sep_2018_2019_magnitude",
        "change_oct": "change_oct_2018_2019_magnitude",
        "change_nov": "change_nov_2018_2019_magnitude",
        "change_dec": "change_dec_2018_2019_magnitude",
        "change_jan": "change_jan_2019_2020_magnitude",
        "change_feb": "change_feb_2019_2020_magnitude",
        "change_mar": "change_mar_2019_2020_magnitude",
        "change_apr": "change_apr_2019_2020_magnitude",
        "change_may": "change_may_2019_2020_magnitude",
        "change_jun": "change_jun_2019_2020_magnitude",
        "change_j_1": "change_jul_2019_2020_magnitude",
        # Percent
        "change_j_2": "change_jul_2018_2019_percent",
        "change_a_1": "change_aug_2018_2019_percent",
        "change_s_1": "change_sep_2018_2019_percent",
        "change_o_1": "change_oct_2018_2019_percent",
        "change_n_1": "change_nov_2018_2019_percent",
        "change_d_1": "change_dec_2018_2019_percent",
        "change_j_3": "change_jan_2019_2020_percent",
        "change_f_1": "change_feb_2019_2020_percent",
        "change_m_1": "change_mar_2019_2020_percent",
        "change_a_2": "change_apr_2019_2020_percent",
        "change_m_2": "change_may_2019_2020_percent",
        "change_j_4": "change_jun_2019_2020_percent",
        "change_j_5": "change_jul_2019_2020_percent",
    }
)

hexagon_grid_gdf = hexagon_grid_gdf.rename(
    columns={
        # Magnitude
        "change_jul": "change_jul_2018_2019_magnitude",
        "change_aug": "change_aug_2018_2019_magnitude",
        "change_sep": "change_sep_2018_2019_magnitude",
        "change_oct": "change_oct_2018_2019_magnitude",
        "change_nov": "change_nov_2018_2019_magnitude",
        "change_dec": "change_dec_2018_2019_magnitude",
        "change_jan": "change_jan_2019_2020_magnitude",
        "change_feb": "change_feb_2019_2020_magnitude",
        "change_mar": "change_mar_2019_2020_magnitude",
        "change_apr": "change_apr_2019_2020_magnitude",
        "change_may": "change_may_2019_2020_magnitude",
        "change_jun": "change_jun_2019_2020_magnitude",
        "change_j_1": "change_jul_2019_2020_magnitude",
        # Percent
        "change_j_2": "change_jul_2018_2019_percent",
        "change_a_1": "change_aug_2018_2019_percent",
        "change_s_1": "change_sep_2018_2019_percent",
        "change_o_1": "change_oct_2018_2019_percent",
        "change_n_1": "change_nov_2018_2019_percent",
        "change_d_1": "change_dec_2018_2019_percent",
        "change_j_3": "change_jan_2019_2020_percent",
        "change_f_1": "change_feb_2019_2020_percent",
        "change_m_1": "change_mar_2019_2020_percent",
        "change_a_2": "change_apr_2019_2020_percent",
        "change_m_2": "change_may_2019_2020_percent",
        "change_j_4": "change_jun_2019_2020_percent",
        "change_j_5": "change_jul_2019_2020_percent",
    }
)

# Data Processing

# Data Post-Processing

# Data Visualization

## Level 1 Boundaries

In [ ]:
# Plot and export July 2018-2019 change
stl.plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_jul_2018_2019_magnitude",
    percent_change="change_jul_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, July 2018 to July 2019",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-jul-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export August 2018-2019 change
stl.plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_aug_2018_2019_magnitude",
    percent_change="change_aug_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, August 2018 to August 2019",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-aug-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export September 2018-2019 change
stl.plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_sep_2018_2019_magnitude",
    percent_change="change_sep_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, September 2018 to September 2019",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-sep-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export October 2018-2019 change
stl.plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_oct_2018_2019_magnitude",
    percent_change="change_oct_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapre Nitrogen Dioxide Change, October 2018 to October 2019",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-oct-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export November 2018-2019 change
stl.plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_nov_2018_2019_magnitude",
    percent_change="change_nov_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, November 2018 to November 2019",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-nov-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export December 2018-2019 change
stl.plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_dec_2018_2019_magnitude",
    percent_change="change_dec_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="South Korea Nitrogen Dioxide Change, December 2018 to December 2019",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-dec-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export January 2019-2020 change
stl.plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_jan_2019_2020_magnitude",
    percent_change="change_jan_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, January 2019 to January 2020",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-jan-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export February 2019-2020 change
stl.plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_feb_2019_2020_magnitude",
    percent_change="change_feb_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, February 2019 to February 2020",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-feb-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export March 2019-2020 change
stl.plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_mar_2019_2020_magnitude",
    percent_change="change_mar_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, March 2019 to March 2020",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-mar-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export April 2019-2020 change
stl.plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_apr_2019_2020_magnitude",
    percent_change="change_apr_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, April 2019 to April 2020",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-apr-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export May 2019-2020 change
stl.plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_may_2019_2020_magnitude",
    percent_change="change_may_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, May 2019 to May 2020",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
stl.outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-may-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export June 2019-2020 change
stl.plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_jun_2019_2020_magnitude",
    percent_change="change_jun_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, June 2019 to June 2020",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-jun-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export July 2019-2020 change
stl.plot_aggregate_change(
    aggregated_data=level_1_gdf,
    magnitude_change="change_jul_2019_2020_magnitude",
    percent_change="change_jul_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=False,
    super_title="Singapore Nitrogen Dioxide Change, July 2019 to July 2020",
    data_source="European Space Agency",
    figsize=(20, 6.5),
)

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-level-1-mean-change-jul-2019-2020.png",
)

rd.save_figure(outpath)

## Hexagon Grid

In [ ]:
# Plot and export July 2018-2019 change
fig, ax = stl.plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_jul_2018_2019_magnitude",
    percent_change="change_jul_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, July 2018 to July 2019",
    data_source="European Space Agency",
)

# Add Malaysia boundary
for axis in [fig.axes[0], fig.axes[1]]:
    malaysia_level_1_clipped.plot(
        ax=axis, facecolor="None", edgecolor="black", linewidth=0.5
    )

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-jul-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export August 2018-2019 change
fig, ax = stl.plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_aug_2018_2019_magnitude",
    percent_change="change_aug_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, August 2018 to August 2019",
    data_source="European Space Agency",
)

# Add Malaysia boundary
for axis in [fig.axes[0], fig.axes[1]]:
    malaysia_level_1_clipped.plot(
        ax=axis, facecolor="None", edgecolor="black", linewidth=0.5
    )

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-aug-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export September 2018-2019 change
fig, ax = stl.plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_sep_2018_2019_magnitude",
    percent_change="change_sep_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, September 2018 to September 2019",
    data_source="European Space Agency",
)

# Add Malaysia boundary
for axis in [fig.axes[0], fig.axes[1]]:
    malaysia_level_1_clipped.plot(
        ax=axis, facecolor="None", edgecolor="black", linewidth=0.5
    )

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-sep-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export October 2018-2019 change
fig, ax = stl.plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_oct_2018_2019_magnitude",
    percent_change="change_oct_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, October 2018 to October 2019",
    data_source="European Space Agency",
)

# Add Malaysia boundary
for axis in [fig.axes[0], fig.axes[1]]:
    malaysia_level_1_clipped.plot(
        ax=axis, facecolor="None", edgecolor="black", linewidth=0.5
    )

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-oct-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export November 2018-2019 change
fig, ax = stl.plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_nov_2018_2019_magnitude",
    percent_change="change_nov_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, November 2018 to November 2019",
    data_source="European Space Agency",
)

# Add Malaysia boundary
for axis in [fig.axes[0], fig.axes[1]]:
    malaysia_level_1_clipped.plot(
        ax=axis, facecolor="None", edgecolor="black", linewidth=0.5
    )

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-nov-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export December 2018-2019 change
fig, ax = stl.plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_dec_2018_2019_magnitude",
    percent_change="change_dec_2018_2019_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, December 2018 to December 2019",
    data_source="European Space Agency",
)

# Add Malaysia boundary
for axis in [fig.axes[0], fig.axes[1]]:
    malaysia_level_1_clipped.plot(
        ax=axis, facecolor="None", edgecolor="black", linewidth=0.5
    )

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-dec-2018-2019.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export January 2019-2020 change
fig, ax = stl.plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_jan_2019_2020_magnitude",
    percent_change="change_jan_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, January 2019 to January 2020",
    data_source="European Space Agency",
)

# Add Malaysia boundary
for axis in [fig.axes[0], fig.axes[1]]:
    malaysia_level_1_clipped.plot(
        ax=axis, facecolor="None", edgecolor="black", linewidth=0.5
    )

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-jan-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export February 2019-2020 change
fig, ax = stl.plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_feb_2019_2020_magnitude",
    percent_change="change_feb_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, February 2019 to February 2020",
    data_source="European Space Agency",
)

# Add Malaysia boundary
for axis in [fig.axes[0], fig.axes[1]]:
    malaysia_level_1_clipped.plot(
        ax=axis, facecolor="None", edgecolor="black", linewidth=0.5
    )

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-feb-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export March 2019-2020 change
fig, ax = stl.plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_mar_2019_2020_magnitude",
    percent_change="change_mar_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, March 2019 to March 2020",
    data_source="European Space Agency",
)

# Add Malaysia boundary
for axis in [fig.axes[0], fig.axes[1]]:
    malaysia_level_1_clipped.plot(
        ax=axis, facecolor="None", edgecolor="black", linewidth=0.5
    )

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-mar-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export April 2019-2020 change
fig, ax = stl.plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_apr_2019_2020_magnitude",
    percent_change="change_apr_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, April 2019 to April 2020",
    data_source="European Space Agency",
)

# Add Malaysia boundary
for axis in [fig.axes[0], fig.axes[1]]:
    malaysia_level_1_clipped.plot(
        ax=axis, facecolor="None", edgecolor="black", linewidth=0.5
    )

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-apr-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export May 2019-2020 change
fig, ax = stl.plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_may_2019_2020_magnitude",
    percent_change="change_may_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, May 2019 to May 2020",
    data_source="European Space Agency",
)

# Add Malaysia boundary
for axis in [fig.axes[0], fig.axes[1]]:
    malaysia_level_1_clipped.plot(
        ax=axis, facecolor="None", edgecolor="black", linewidth=0.5
    )

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-may-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export June 2019-2020 change
fig, ax = stl.plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_jun_2019_2020_magnitude",
    percent_change="change_jun_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, June 2019 to June 2020",
    data_source="European Space Agency",
)

# Add Malaysia boundary
for axis in [fig.axes[0], fig.axes[1]]:
    malaysia_level_1_clipped.plot(
        ax=axis, facecolor="None", edgecolor="black", linewidth=0.5
    )

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-jun-2019-2020.png",
)

rd.save_figure(outpath)

In [ ]:
# Plot and export July 2019-2020 change
fig, ax = stl.plot_aggregate_change(
    aggregated_data=hexagon_grid_gdf,
    magnitude_change="change_jul_2019_2020_magnitude",
    percent_change="change_jul_2019_2020_percent",
    vector_boundary=singapore_level_1_geometry,
    plot_vector=True,
    super_title="Singapore Nitrogen Dioxide Change, July 2019 to July 2020",
    data_source="European Space Agency",
)

# Add Malaysia boundary
for axis in [fig.axes[0], fig.axes[1]]:
    malaysia_level_1_clipped.plot(
        ax=axis, facecolor="None", edgecolor="black", linewidth=0.5
    )

# Export figure
outpath = os.path.join(
    "04-graphics-outputs",
    "spatial-plots",
    "singapore",
    "singapore-no2-aggregate-hexagon-grid-mean-change-jul-2019-2020.png",
)

rd.save_figure(outpath)

# Data Export